In [314]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [315]:
import numpy as np

import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [316]:
def get_vertices_stirling(Th, Tc, v_max, v_min):
    v = [v_max, v_min, v_min, v_max]
    p = [N*R*Tc/v[0], N*R*Tc/v[1], N*R*Th/v[2], N*R*Th/v[3]]
    return v, p    

In [317]:
def get_vertices_diesel(Th, Tc, v_max, v_min):
    v = [v_max, v_min, 0.0, v_max]
    p = [N*R*Tc/v[0], 0.0, 0.0, 0.0]
    
    p[1] = p[0] * (v[0]/v[1])**gamma
    p[2] = p[1]
    v[2] = N*R*Th/p[2]
    p[3] = p[2] * (v[2]/v[3])**gamma
    
    return v, p

In [318]:
def get_vertices_carnot(Th, Tc, v_max, v_min):
    v = [v_max, 0.0, v_min, 0.0]
    p = [N*R*Tc/v[0], 0.0, N*R*Th/v[2], 0.0]
    
    v[1] = (v[2]**gamma/v[0] * p[2]/p[0])**(1.0/(gamma-1.0))
    p[1] = N*R*Tc/v[1]
    v[3] = (v[0]**gamma/v[2] * p[0]/p[2])**(1.0/(gamma-1.0))                
    p[3] = N*R*Th/v[3]
    
    return v, p

In [319]:
def get_path(vi,pi,vf,pf):

    v_values = np.linspace(vi,vf,pts)
    p_values = np.empty((pts))
    
    if abs(vf-vi) < 0.01:  # Isochoric case
        p_values = np.linspace(pi,pf,pts)
    else:
        j = np.log(pi/pf)/np.log(vf/vi)
        for i in range(pts): 
            p_values[i] = pi * (vi/v_values[i])**j

    return v_values, p_values

In [320]:
def get_isotherm(T):
    
    v_values = np.linspace(v_min, v_max, pts)
    p_values = np.empty((pts))
    
    for i in range(pts):
        p_values[i]  = N*R*T/v_values[i]
            
    return v_values, p_values
    

In [321]:
def get_work(v_values, p_values):


    W = 0.0
    dv = (v_values[pts-1] - v_values[0]) / pts
    for i  in range(pts):
        v = v_values[i]
        p = p_values[i]
        W = W - p*dv
    W = C*W    # Convert W form atm*L to J
    return W

In [322]:
def get_energy_change(vi, pi, vf, pf, gamma):

    Cv = 1.0 / (gamma-1.0) #Adimensional value of Cv (Cv/NR indeed)
    dU = C * Cv*(vf*pf - vi*pi) # Energy difference in Joules
    return dU

In [323]:
def get_cycle(v_vertices, p_vertices):
    x_values = np.empty((4,pts))
    y_values = np.empty((4,pts))
    for i in range(3):  
        v_values, p_values = get_path(v_vertices[i], p_vertices[i], v_vertices[i+1], p_vertices[i+1])
        x_values[i] = v_values
        y_values[i] = p_values
    
    v_values, p_values = get_path(v_vertices[3], p_vertices[3], v_vertices[0], p_vertices[0])
    x_values[3] = v_values
    y_values[3] = p_values

    return x_values, y_values

In [324]:
def get_process(vi,pi,j):

    

    v_values = np.linspace(v_min, v_max, pts)
    p_values = np.empty((pts))
    for i in range(pts):
        p_values[i] = pi* (vi/v_values[i])**j
        if p_values[i] > 2*p_max:                    # This sentence truncates high y_values to prevent memory overflow
            p_values[i] = 2*p_max
            
    return v_values, p_values

In [325]:
def update_volume_labels(process):
    
    if process == 'Stirling':
        volume_text.value = 'Isokoroen balioak'
    elif process == 'Diesel':
        volume_text.value = 'Prozesuaren mugak'
    elif process == 'Carnot':
        volume_text.value = 'Prozesuaren mugak'

In [326]:
def update_energies(x_values, y_values):
    Q_h = 0.0
    Q_c = 0.0
    W = 0.0
    colors = ['blue', 'blue', 'blue', 'blue']
    for i in range(4):
        v_values = x_values[i]
        p_values = y_values[i]
        W_process = get_work(v_values, p_values)
        dU_process = get_energy_change(v_values[0], p_values[0], v_values[pts-1], p_values[pts-1], gamma)
        Q_process = dU_process - W_process
        
        
        W = W + W_process        
        if i == 0 or i == 3:
            Q_c = Q_c + Q_process
        else:
            Q_h = Q_h + Q_process
        
        if abs(Q_process) > 10.0 and  abs(dU_process) > 10.0:
            colors[i] = 'red'
            
    Q_h = abs(Q_h)
    Q_c = abs(Q_c)
    W = abs(W)
    eta = W/Q_h
    
    Qh_text.value = '%.0f' % Q_h
    Qc_text.value ='%.0f' % Q_c
    W_text.value ='%.0f' % W
    eta_text.value = '%.3f' % eta
    cycle.colors = colors

In [327]:
def update_cycle(v_vertices, p_vertices, process):
    
    x_values, y_values = get_cycle(v_vertices, p_vertices)  
    cycle.x = x_values
    cycle.y = y_values

    update_energies(x_values, y_values)
    

In [328]:
def update_points(change):
    vi = vi_slider.value
    vf = vf_slider.value
    Tc = Tc_slider.value
    Th = Th_slider.value
    process = process_dropdown.value
    
    if process == 'Stirling':
        v_vertices, p_vertices = get_vertices_stirling(Th, Tc, vf, vi)
    elif process == 'Diesel':
        v_vertices, p_vertices = get_vertices_diesel(Th, Tc, vf, vi)     
    elif process == 'Carnot':
        v_vertices, p_vertices = get_vertices_carnot(Th, Tc, vf, vi)
    vertices.x = v_vertices
    vertices.y = p_vertices

    

    update_cycle(v_vertices, p_vertices, process)
    update_volume_labels(process)

In [329]:
def update_isotherms(change):
    Th = Th_slider.value
    Tc = Tc_slider.value
    
    x_values = np.empty((2,pts))
    y_values = np.empty((2,pts))
    
    v_values, p_values = get_isotherm(Th)
    x_values[0] = v_values
    y_values[0] = p_values
    
    v_values, p_values = get_isotherm(Tc)
    x_values[1] = v_values
    y_values[1] = p_values
    
    isotherms.x = x_values
    isotherms.y = y_values
    
    update_points(None)

In [330]:
#######################
###   PARAMETERS    ###
#######################

## Global Parameters

N = 1.0
R = 0.082057 # Ideal gas constant in atm*L/mol/K
C = 101.325 # Conversion factor from atm*L to J -> C = J/atmL
pts = 2000 # Number of points for plotting each process
gamma = 5.0/3.0

## Limits of parameters (volumes in L, pressures in atm)

# Limits of the figure
v_min = 0.01
v_max = 10.0
p_min = 0.01
p_max = 20.0

# Limits of the sliders
Th_min = 300.0
Th_max = 500.0
Tc_min = 200.0
Tc_max = 300.0
v_min_lowerbound = 1.0
v_min_upperbound = 3.0
v_max_lowerbound = 5.0
v_max_upperbound = 8.0

# Initial values
vi = 3.0
vf = 5.0
Tc = 200.0
Th = 400.0
gamma = 5.0/3.0 # Adiabatic index (5/3 for monoatomic gases, 7/5 for diatomic gases at room temperature)
N = 1.0 # Number of mols of gas

########################
###CREATE THE FIGURES###
########################

fig_125_001 = bq.Figure(title='Prozesu politropikoak',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout=widgets.Layout(width='100%')
    )


scale_x = bqs.LinearScale(min = v_min, max = v_max)
scale_y = bqs.LinearScale(min = p_min, max = p_max)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(v_min, v_max, 11),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v (L)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values= np.linspace(p_min, p_max, 6),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_125_001.axes = [axis_x,axis_y]

########################
####CREATE THE MARKS####
########################

cycle = bqm.Lines(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y},
                display_legend=False,
                colors = ['blue'],
)

isotherms = bqm.Lines(
                x = [], 
                y = [], 
                scales = {'x': scale_x, 'y': scale_y},
                display_legend=True,
                labels = ["Foku beroa", "Foku hotza"],
                colors = ['#FF0000', '#FF7700'],
                opacities = [0.3,0.3]
)

points = bqm.Scatter(
    x = [],
    y = [],
    scales = {'x': scale_x, 'y': scale_y}, 
    opacities = [1.0],
    visible = True,
    colors = ['green'],
    labels=['Hasierako egoera'],
)      

vertices = bqm.Scatter(
    x = [],
    y = [],
    scales = {'x': scale_x, 'y': scale_y}, 
    opacities = [0.0],
    visible = True,
    colors = ['green'],
    labels=['Hasierako egoera'],
) 
fig_125_001.marks = [cycle, isotherms, points, vertices, cycle]

########################
######  WIDGETS  #######
########################

Th_slider = widgets.FloatSlider(
    value=Th,
    min=Th_min,
    max=Th_max,
    step=0.1,
    description='$T_H$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

Th_slider.observe(update_isotherms, 'value')

Tc_slider = widgets.FloatSlider(
    value=Tc,
    min=Tc_min,
    max=Tc_max,
    step=0.1,
    description='$T_C$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

Tc_slider.observe(update_isotherms, 'value')

process_dropdown = widgets.Dropdown(
    options=['Stirling', 'Diesel', 'Carnot'],
    value='Stirling',
    description='Zikloa',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

process_dropdown.observe(update_points, 'value')

# Labels:

Qh_text = widgets.Label(value='')
Qc_text = widgets.Label(value='')
W_text = widgets.Label(value='')
eta_text = widgets.Label(value='')

#process_dropdown.observe(update_figure, 'value')

volume_text = widgets.Label(value='')

vi_slider = widgets.FloatSlider(
    value=vi,
    min=v_min_lowerbound,
    max=v_min_upperbound,
    step=0.1,
    description='$v_{min}$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vi_slider.observe(update_points, 'value')

vf_slider = widgets.FloatSlider(
    value=vf,
    min=v_max_lowerbound,
    max=v_max_upperbound,
    step=0.1,
    description='$v_{max}$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vf_slider.observe(update_points, 'value')

########################
######  LAYOUT  ########
########################


# Left Block ##
left_block_125_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
left_block_125_000.children = [Th_slider, Tc_slider, process_dropdown, volume_text, vi_slider, vf_slider]
                              
## Center Block ##
center_block_125_000 = widgets.VBox([], layout=widgets.Layout(width='70%', align_items='center'))
center_block_125_000.children = [fig_125_001]

## Right Block ##
right_block_125_000 = widgets.VBox([], layout=widgets.Layout(width='10%', align_items='center'))
right_block_125_000.children = [
                                widgets.HBox([widgets.Label(value='$Q_h=$'),Qh_text,widgets.Label(value='$J$')], layout=widgets.Layout(width='100%')),
                                widgets.HBox([widgets.Label(value='$Q_c=$'),Qc_text,widgets.Label(value='$J$')], layout=widgets.Layout(width='100%')),
                                widgets.HBox([widgets.Label(value='$W=$'),W_text,widgets.Label(value='$J$')], layout=widgets.Layout(width='100%')),
                                widgets.HBox([widgets.Label(value='$\eta =$'),eta_text], layout=widgets.Layout(width='100%'))
                               ]

## Main Block ##

main_block_125_000 = widgets.HBox([],layout=widgets.Layout(width='100%', align_items='center'))
main_block_125_000.children = [left_block_125_000, center_block_125_000, right_block_125_000]

main_block_125_000